In [30]:
import os
import chardet
import openai

In [31]:
# 假设文件名为'label.txt'
file_path = 'label.txt'

# 初始化一个空列表来存储读取的行
labelline = []

# 打开文件并读取每一行
with open(file_path, 'r') as file:
    for line in file:
        # 去除每行的首尾空白字符，包括换行符
        cleaned_line = line.strip().replace(',', '').replace('"', '')
        # 如果清理后的行不为空，添加到列表中
        if cleaned_line:
            labelline.append(cleaned_line)
labelline = labelline[1:-2]

In [32]:
# 假设文件名为'label.txt'
file_path = 'inputdata.txt'

# 初始化一个空列表来存储读取的行
dataline = []

# 打开文件并读取每一行
with open(file_path, 'r') as file:
    for line in file:
        # 去除每行的首尾空白字符，包括换行符
        cleaned_line = line.strip().replace(',', '').replace('"', '')
        # 如果清理后的行不为空，添加到列表中
        if cleaned_line:
            dataline.append(cleaned_line)
dataline = dataline[1:-2]

In [33]:
import openai
import os
import json
import re
import json

openai.api_key = "0c2fecccc4224abcb5b80f6c69a52663"

system_intel = "You are GPT-4, answer my questions as if you were an expert in the field."
prompt = '''Imagine you are an RFC documentation expert currently working on some functions that do not have corresponding RFC sections indicated, such as:
```cpp
//----- (0000000180022240) ----------------------------------------------------
__int64 __fastcall CTls13ServerContext::TlsParseClientHello(
        CTls13ServerContext *this,
        unsigned __int8 *a2,
        unsigned int a3,
        unsigned __int8 *const a4,
        unsigned int *a5,
        unsigned int **a6,
        unsigned int *a7)
{
  return CTls13ServerHandshake::ParseClientHello((CTls13ServerContext *)((char *)this + 1288), a2, a3, a4, a5, a6, a7);
}
```
It is known that this example function is based on a section of RFC 8446. After reading and analyzing the code, you should output:
4.1.2 Client Hello
And nothing else. You should output this function because the function in the example is strongly related to section 4.1.2. Now, please read this function and provide the corresponding output. Note that your output should belong to one of the sections below and should be as specific as possible, for example, 4.1.2. If unsure, consider a broader section like 4:
```
Table of Contents

Introduction

1.1. Conventions and Terminology
1.2. Major Differences from TLS 1.2
1.3. Updates Affecting TLS 1.2
Protocol Overview

2.1. Incorrect DHE Share
2.2. Resumption and Pre-Shared Key (PSK)
2.3. 0-RTT Data
Presentation Language

3.1. Basic Block Size
3.2. Miscellaneous
3.3. Numbers
3.4. Vectors
3.5. Enumerateds
3.6. Constructed Types
3.7. Constants
3.8. Variants
Handshake Protocol

4.1. Key Exchange Messages
4.1.1. Cryptographic Negotiation
4.1.2. Client Hello
4.1.3. Server Hello
4.1.4. Hello Retry Request
4.2. Extensions
4.2.1. Supported Versions
4.2.2. Cookie
4.2.3. Signature Algorithms
4.2.4. Certificate Authorities
4.2.5. OID Filters
4.2.6. Post-Handshake Client Authentication
4.2.7. Supported Groups
4.2.8. Key Share
4.2.9. Pre-Shared Key Exchange Modes
4.2.10. Early Data Indication
4.2.11. Pre-Shared Key Extension
4.3. Server Parameters
4.3.1. Encrypted Extensions
4.3.2. Certificate Request
4.4. Authentication Messages
4.4.1. The Transcript Hash
4.4.2. Certificate
4.4.3. Certificate Verify
4.4.4. Finished
4.5. End of Early Data
4.6. Post-Handshake Messages
4.6.1. New Session Ticket Message
4.6.2. Post-Handshake Authentication
4.6.3. Key and Initialization Vector Update
Record Protocol

5.1. Record Layer
5.2. Record Payload Protection
5.3. Per-Record Nonce
5.4. Record Padding
5.5. Limits on Key Usage
Alert Protocol

6.1. Closure Alerts
6.2. Error Alerts
Cryptographic Computations

7.1. Key Schedule
7.2. Updating Traffic Secrets
7.3. Traffic Key Calculation
7.4. (EC)DHE Shared Secret Calculation
7.4.1. Finite Field Diffie-Hellman
7.4.2. Elliptic Curve Diffie-Hellman
7.5. Exporters
0-RTT and Anti-Replay

8.1. Single-Use Tickets
8.2. Client Hello Recording
8.3. Freshness Checks
Compliance Requirements

9.1. Mandatory-to-Implement Cipher Suites
9.2. Mandatory-to-Implement Extensions
9.3. Protocol Invariants
Security Considerations

IANA Considerations

References

12.1. Normative References
12.2. Informative References
Appendices

Appendix A. State Machine
A.1. Client
A.2. Server
Appendix B. Protocol Data Structures and Constant Values
B.1. Record Layer
B.2. Alert Messages
B.3. Handshake Protocol
B.3.1. Key Exchange Messages
B.3.2. Server Parameters Messages
B.3.3. Authentication Messages
B.3.4. Ticket Establishment
B.3.5. Updating Keys
B.4. Cipher Suites
Appendix C. Implementation Notes
C.1. Random Number Generation and Seeding
C.2. Certificates and Authentication
C.3. Implementation Pitfalls
C.4. Client Tracking Prevention
C.5. Unauthenticated Operation
Appendix D. Backward Compatibility
D.1. Negotiating with an Older Server
D.2. Negotiating with an Older Client
D.3. 0-RTT Backward Compatibility
D.4. Middlebox Compatibility Mode
D.5. Security Restrictions Related to Backward Compatibility
Appendix E. Overview of Security Properties
E.1. Handshake
E.1.1. Key Derivation and HKDF
E.1.2. Client Authentication
E.1.3. 0-RTT
E.1.4. Exporter Independence
E.1.5. Post-Compromise Security
E.1.6. External References
E.2. Record Layer
E.2.1. External References
E.3. Traffic Analysis
E.4. Side-Channel Attacks
E.5. Replay Attacks on 0-RTT
E.5.1. Replay and Exporters
E.6. PSK Identity Exposure
E.7. Sharing PSKs
E.8. Attacks on Static RSA
Contributors

Author's Address
```
here is your task:
---
'''


import requests
api_key = '0c2fecccc4224abcb5b80f6c69a52663'
url = "https://gpt4-j.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-06-01"
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}
def ask_GPT4(prompt):
    data = {
        "messages": [
      {
       "role": "system",
       "content": "You are GPT-4, answer my questions as if you were an expert in the field."
      },
      {
       "role": "user",
       "content": prompt
      }
     ],
     "temperature" : 0,
    }

    response = requests.post(url, headers=headers, json=data)
    response = response.json()
    if 'choices' in response:
      return response['choices'][0]['message']['content']
    else:
      return response
    


In [34]:
analysis_result = []
a = 1

for data in dataline:
    if(a < 5):
        analysis_result.append(ask_GPT4(prompt+data+"\n---"))
    a += 1

In [35]:
# 获取analysis_result的长度
num_items = len(analysis_result)

# 只取labelline的前num_items项进行比较
labelline_to_compare = labelline[:num_items]

# 使用列表推导式比较对应项是否一致
comparison_results = [a == b for a, b in zip(analysis_result, labelline_to_compare)]

# 检查是否有不一致的情况
if all(comparison_results):
    print("所有对应项都一致。")
else:
    print("存在不一致的对应项。")

    # 找出不一致的项
    inconsistent_items = [(i, analysis_result[i], labelline_to_compare[i]) for i in range(num_items) if not comparison_results[i]]
    for index, result, label in inconsistent_items:
        print(f"不一致的项在索引 {index}: analysis_result 中是 '{result}', labelline 中是 '{label}'")

存在不一致的对应项。
不一致的项在索引 0: analysis_result 中是 '4.1.2 Client Hello', labelline 中是 '4.1.2. Client Hello'
不一致的项在索引 1: analysis_result 中是 '4.2.4 Certificate Authorities', labelline 中是 '4.4.2. Certificate'
不一致的项在索引 2: analysis_result 中是 '4.2.8 Key Share', labelline 中是 '4.2.8. Key Share'
不一致的项在索引 3: analysis_result 中是 '{'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-06-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 51 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}', labelline 中是 '4.1.2. Client Hello'
